In [ ]:
#filter(like) items  와는 다르게 groupby 에 쓴다

In [ ]:
df.groupby('').filter(함수)

# filter()
- `DataFrameGroupBy.filter(func, dropna=True, *args, **kwargs)`
- 특정 집계 조건을 만족하는 Group의 행들만 조회한다.
    1. DataFrameGroupBy의 group로 DataFrame을 함수에 전달한다.
    2. 함수는 받은 DataFrame을 이용해 집계한 값의 조건을 비교해서 반환한다.(반환타입: Bool) 
    3. 반환값이 True인 Group들의 모든 행들로 구성된 DataFrame을 반환한다.
- 매개변수
    - func: filtering 조건을 구현한 함수
        - 첫번째 매개변수로 Group으로 묶인 DataFrame을 받는다.
    - dropna=True
        - 필터를 통과하지 못한 group의 DataFrame의 값들을 drop시킨다. False로 설정하면 NA 처리해서 반환한다.
    - \*args, \*\*kwargs: filter 함수의 매개변수에 전달할 전달인자값.

In [187]:
import numpy as np
import pandas as pd

In [188]:
# cnt1 - 사과: 10대, 귤: 20대, 배: 단단위, 딸기 30이상
data = dict(fruits=['사과', '사과','사과', '사과','사과','귤','귤','귤','귤','귤','배','배','배','배','배','딸기','딸기','딸기','딸기','딸기']
            ,cnt1=[10, 12, 13, 11, 12, 21, 22, 27, 24, 26, 7, 7, 8, 3, 2, 30, 35, 37, 41, 28]
            ,cnt2=[100,  103, 107, 107,  101,  51,  57, 58,  57, 51,  9, 9,  5,  7,  7,  208, 217, 213, 206, 204]
           )
df = pd.DataFrame(data)
df

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51


In [12]:
df.filter(like='cnt')  # DataFrame 에서 그냥 컬럼 조회 할때 쓰는것

,cnt1,cnt2
0,10,100
1,12,103
2,13,107
3,11,107
4,12,101
5,21,51
6,22,57
7,27,58
8,24,57
9,26,51


In [6]:
r = df.groupby('fruits')['cnt1'].mean()
r[r >=20]  # 집계결과만 

fruits
귤     24.0
딸기    34.2
Name: cnt1, dtype: float64

#### 과일중 cnt1의 평균이 20 이상인 과일들만 보기
- 집계결과가 아니라 조건을 만족하는 행들을 다 출력
- aggregate, agg ==>함수를 그냥 사용하는것 / filter는 함수를 넣어 블리언값으로 true인 그룹의 모든 행을 출력

In [10]:
# True False 의 기준이 되는 함수 // 매개변수 groupbydataFrame
def check_cnt1_mean(x):
    """
    [매개변수]
        x : DataFrame(group별로 나눈 DataFrame)
    [반환값]
        bool :x['cnt1']의 평균이 20 이상인지 여부
    """
    return x['cnt1'].mean() >=20

In [78]:
lambda x:x['cnt1'].mean()>=20

<function __main__.<lambda>(x)>

In [11]:
df.groupby('fruits').filter(check_cnt1_mean)

,fruits,cnt1,cnt2
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [ ]:
#람다 표현식 => 간단한 함수를 만드닌 식.
#lambda[매개변수]:처리식
#처리식의 결과를 알아서 return 한다.

In [13]:
df.groupby('fruits').filter(lambda x :x['cnt1'].mean() >=20)

,fruits,cnt1,cnt2
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [14]:
#dropna = False: 조건이 False인 행들도 NaN 처리해서 출력해준다.
df.groupby('fruits').filter(lambda x :x['cnt1'].mean() >=20, dropna=False)

,fruits,cnt1,cnt2
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,귤,21.0,51.0
6,귤,22.0,57.0
7,귤,27.0,58.0
8,귤,24.0,57.0
9,귤,26.0,51.0


#### 매개변수 있는 filter함수

In [15]:
def check_mean(x, col, threshold):
    """
    [매개변수]
        x =DataFrame
        col : str 평균을 구할 컬럼명
        threshodl: int 비교대상값
    """
    return x[col].mean() >= threshold

In [17]:
df.groupby('fruits').filter(check_mean, col = 'cnt1' ,threshold = 20)

,fruits,cnt1,cnt2
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [19]:
r = df.groupby('fruits')['cnt1'].mean()
r[r >=20]  # 집계결과만 

fruits
귤     24.0
딸기    34.2
Name: cnt1, dtype: float64

In [18]:
df.groupby('fruits').filter(check_mean, col = 'cnt1' ,threshold = 30)

,fruits,cnt1,cnt2
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [20]:
df.groupby('fruits').filter(check_mean, col = 'cnt2' ,threshold = 100)

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [191]:
# 람다식을 사용할 경우는 추가 매개변수는 코드상에 직접 값으로 설정하고 작성하면 된다.
# 항상동일한 값이 전달 되므로 
# df.groupby('fruits').filter(lambda x, col, threshold : x [col].mean()>=threshold,
#                                 col = 'cnt1',
#                               threshold=30)  
df.groupby('fruits').filter(lambda x: x['cnt1'].mean()>=30)
#                               

,fruits,cnt1,cnt2
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


# transform
함수에 의해 처리된 값(반환값)으로 원래 값들을 변경(tranform) 해서 반환    
DataFrame에 Group 단위 통계량을 추가할 때 유용하다.
- `DataFrameGroupBy.transform(func, *args)`, `SeriesGroupBy..transform(func, *args)`
    - func: 매개변수로 그룹별로 Series를 받아 Series의 값들을 변환하여 (Series로)반환하는 함수객체
        - DataFrameGroupBy은 모든 컬럼의 값들을 group 별 Series로 전달한다.
    - *args: 함수에 전달할 추가 인자값이 있으면 매개변수 순서에 맞게 값을 전달한다. (위치기반 argument)
- transform() 함수를 groupby() 와 사용하면 컬럼의 각 원소들을 자신이 속한 그룹의 통계량으로 변환된 데이터셋을 생성할 수 있다.
- 컬럼의 값과 통계값을 비교해서 보거나 결측치 처리등에 사용할 수있다.

In [23]:
 df.groupby('fruits').mean()


,cnt1,cnt2
fruits,,
귤,24.0,54.8
딸기,34.2,209.6
배,5.4,7.4
사과,11.6,103.6


In [26]:
df.shape

(20, 3)

In [79]:
r=df.groupby('fruits').transform('mean')
print(r.shape) # 그룹으로 묶인 컬럼은 빠진다.fruits 컬럼
print(r)

(20, 2)
    cnt1   cnt2
0   11.6  103.6
1   11.6  103.6
2   11.6  103.6
3   11.6  103.6
4   11.6  103.6
5   24.0   54.8
6   24.0   54.8
7   24.0   54.8
8   24.0   54.8
9   24.0   54.8
10   5.4    7.4
11   5.4    7.4
12   5.4    7.4
13   5.4    7.4
14   5.4    7.4
15  34.2  209.6
16  34.2  209.6
17  34.2  209.6
18  34.2  209.6
19  34.2  209.6


In [27]:
def func(x):
    """
    [매개변수]
    x: Series - 그룹별 각 컬럼들을 SEries로 전달
    [반환값]
        int :max - min
    """
    return x.max()-x.min()

In [29]:
df.groupby('fruits').transform(func)

,cnt1,cnt2
0,3,7
1,3,7
2,3,7
3,3,7
4,3,7
5,6,7
6,6,7
7,6,7
8,6,7
9,6,7


In [30]:
df.groupby('fruits').transform(lambda x:x.max() - x.min())

,cnt1,cnt2
0,3,7
1,3,7
2,3,7
3,3,7
4,3,7
5,6,7
6,6,7
7,6,7
8,6,7
9,6,7


In [33]:
# 컬럼단위로 처리.
df.groupby('fruits')['cnt1'].transform('mean')

0     11.6
1     11.6
2     11.6
3     11.6
4     11.6
5     24.0
6     24.0
7     24.0
8     24.0
9     24.0
10     5.4
11     5.4
12     5.4
13     5.4
14     5.4
15    34.2
16    34.2
17    34.2
18    34.2
19    34.2
Name: cnt1, dtype: float64

In [35]:
 df.groupby('fruits').mean()

,cnt1,cnt2
fruits,,
귤,24.0,54.8
딸기,34.2,209.6
배,5.4,7.4
사과,11.6,103.6


#### 원본에 통계치 붙여서 비교하기

In [38]:
# 그룹별 평균을 Transform 을 이용해 조회
cnt1_group_mean = df.groupby('fruits')['cnt1'].transform('mean')
print(type(cnt1_group_mean))
cnt1_group_mean

<class 'pandas.core.series.Series'>


0     11.6
1     11.6
2     11.6
3     11.6
4     11.6
5     24.0
6     24.0
7     24.0
8     24.0
9     24.0
10     5.4
11     5.4
12     5.4
13     5.4
14     5.4
15    34.2
16    34.2
17    34.2
18    34.2
19    34.2
Name: cnt1, dtype: float64

In [43]:
cnt2_group_median = df.groupby('fruits')['cnt2'].transform('median')
cnt2_group_median

0     103
1     103
2     103
3     103
4     103
5      57
6      57
7      57
8      57
9      57
10      7
11      7
12      7
13      7
14      7
15    208
16    208
17    208
18    208
19    208
Name: cnt2, dtype: int64

In [41]:
# insert( ) : 새로운 컬럼을 중간에 넣는다.
df2 = df.copy()
df2.insert(2,'cnt1 평균',cnt1_group_mean)
# 원본을 바꿈

In [42]:
df2

,fruits,cnt1,cnt1 평균,cnt2
0,사과,10,11.6,100
1,사과,12,11.6,103
2,사과,13,11.6,107
3,사과,11,11.6,107
4,사과,12,11.6,101
5,귤,21,24.0,51
6,귤,22,24.0,57
7,귤,27,24.0,58
8,귤,24,24.0,57
9,귤,26,24.0,51


In [46]:
df2['cnt2 중앙값'] = cnt2_group_median
df2

,fruits,cnt1,cnt1 평균,cnt2,cnt2 중앙값
0,사과,10,11.6,100,103
1,사과,12,11.6,103,103
2,사과,13,11.6,107,103
3,사과,11,11.6,107,103
4,사과,12,11.6,101,103
5,귤,21,24.0,51,57
6,귤,22,24.0,57,57
7,귤,27,24.0,58,57
8,귤,24,24.0,57,57
9,귤,26,24.0,51,57


In [55]:
# 행을 섞기. 
#df3 = 
df3 = df.sample(frac = 1).reset_index(drop=True) # 전체 데이터 중에 50% 만 섞은 것을 반환
df3.head()


,fruits,cnt1,cnt2
0,사과,12,103
1,딸기,41,206
2,귤,24,57
3,배,3,7
4,딸기,37,213


### 이런상황에서 transform 을 많이쓴다.

In [58]:
c2 = df3.groupby('fruits')['cnt2'].transform('mean')  # 
df3['cnt2 평균'] = c2
df3

,fruits,cnt1,cnt2,cnt2 평균
0,사과,12,103,103.6
1,딸기,41,206,209.6
2,귤,24,57,54.8
3,배,3,7,7.4
4,딸기,37,213,209.6
5,배,7,9,7.4
6,딸기,30,208,209.6
7,배,8,5,7.4
8,사과,12,101,103.6
9,귤,26,51,54.8


#### 결측치 처리
- transform이용해서 여기선 결측치를 같은 "과일그룹의 평균값"으로 변환
    - 전체 평균보다 좀더 정확할 수 있다.

In [ ]:
# 결측치 변경(대체)
# series.fillna(대체할 값)
# 대체값 : scalar - 동일한 값으로 대체
# 대체값 : 배열형태 (리스트, 시리즈, ndarray) - NA의 index와 동일한 index에 있는 값으로 대체

In [60]:
s = pd.Series([1,np.nan,3,np.nan,5])
rv = pd.Series([10,20,30,40,50])


In [61]:
s.fillna(1000)

0       1.0
1    1000.0
2       3.0
3    1000.0
4       5.0
dtype: float64

In [62]:
s.fillna(rv)  # s와 rv의 size는 동일 

0     1.0
1    20.0
2     3.0
3    40.0
4     5.0
dtype: float64

In [64]:
df4 = df.copy()
df4.head()

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101


In [67]:
df4.loc[[0,1,5,6,10,11,15,16], 'cnt2'] = np.nan  # 일괄적으로 결측치로 바꿈
df4

,fruits,cnt1,cnt2
0,사과,10,NaN
1,사과,12,NaN
2,사과,13,107.0
3,사과,11,107.0
4,사과,12,101.0
5,귤,21,NaN
6,귤,22,NaN
7,귤,27,58.0
8,귤,24,57.0
9,귤,26,51.0


In [69]:
# 결측치 제거 
df4.dropna()  # 결측치 있는 행자체를 제거 

,fruits,cnt1,cnt2
2,사과,13,107.0
3,사과,11,107.0
4,사과,12,101.0
7,귤,27,58.0
8,귤,24,57.0
9,귤,26,51.0
12,배,8,5.0
13,배,3,7.0
14,배,2,7.0
17,딸기,37,213.0


In [71]:
df4['cnt2'].fillna(df4['cnt2'].mean())  #cnt2 전체의 평균으로 nan 값을 대체했다.

0      93.583333
1      93.583333
2     107.000000
3     107.000000
4     101.000000
5      93.583333
6      93.583333
7      58.000000
8      57.000000
9      51.000000
10     93.583333
11     93.583333
12      5.000000
13      7.000000
14      7.000000
15     93.583333
16     93.583333
17    213.000000
18    206.000000
19    204.000000
Name: cnt2, dtype: float64

In [73]:
# 과일별 cnt2의 평균으로 결측치 대체
c2_mean = df4.groupby('fruits')['cnt2'].transform('mean')
c2_mean

0     105.000000
1     105.000000
2     105.000000
3     105.000000
4     105.000000
5      55.333333
6      55.333333
7      55.333333
8      55.333333
9      55.333333
10      6.333333
11      6.333333
12      6.333333
13      6.333333
14      6.333333
15    207.666667
16    207.666667
17    207.666667
18    207.666667
19    207.666667
Name: cnt2, dtype: float64

In [77]:
df4['cnt2'].fillna(c2_mean)

0     105.000000
1     105.000000
2     107.000000
3     107.000000
4     101.000000
5      55.333333
6      55.333333
7      58.000000
8      57.000000
9      51.000000
10      6.333333
11      6.333333
12      5.000000
13      7.000000
14      7.000000
15    207.666667
16    207.666667
17    213.000000
18    206.000000
19    204.000000
Name: cnt2, dtype: float64

In [76]:
df4['cnt2']= df4['cnt2'].fillna(c2_mean)
df4

,fruits,cnt1,cnt2
0,사과,10,105.000000
1,사과,12,105.000000
2,사과,13,107.000000
3,사과,11,107.000000
4,사과,12,101.000000
5,귤,21,55.333333
6,귤,22,55.333333
7,귤,27,58.000000
8,귤,24,57.000000
9,귤,26,51.000000


In [80]:
df

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51


# TODO 
- data/diamonds.csv 조회

In [227]:
import pandas as pd
import numpy as np

In [192]:
dia=pd.read_csv('data/diamonds.csv')

In [195]:
dia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.1+ MB


In [198]:
dia['cut'].unique()

array(['Ideal', 'Premium', 'Good', 'Very Good', 'Fair'], dtype=object)

In [228]:
dia.groupby('cut')['price'].mean().sort_values()

cut
Ideal        3457.541970
Good         3928.864452
Very Good    3981.759891
Fair         4358.757764
Premium      4584.257704
Name: price, dtype: float64

In [229]:
# cut 별 평균 가격이 4000 이상인 diamond 데이터들 조회 
result = dia.groupby('cut').filter(lambda x : x['price'].mean() >=4000)
result

,carat,cut,color,clarity,clarity_mean_price,depth,table,price,x,y,z
1,0.21,Premium,E,SI1,NaN,59.8,61.0,326,3.89,3.84,2.31
3,0.29,Premium,I,VS2,NaN,62.4,58.0,334,4.20,4.23,2.63
8,0.22,Fair,E,VS2,NaN,65.1,61.0,337,3.87,3.78,2.49
12,0.22,Premium,F,SI1,NaN,60.4,61.0,342,3.88,3.84,2.33
14,0.20,Premium,E,SI2,NaN,60.2,62.0,345,3.79,3.75,2.27
15,0.32,Premium,E,I1,NaN,60.9,58.0,345,4.38,4.42,2.68
26,0.24,Premium,I,VS1,NaN,62.5,57.0,355,3.97,3.94,2.47
45,0.29,Premium,F,SI1,NaN,62.4,58.0,403,4.24,4.26,2.65
53,0.22,Premium,E,VS2,NaN,61.6,58.0,404,3.93,3.89,2.41
54,0.22,Premium,D,VS2,NaN,59.3,62.0,404,3.91,3.88,2.31


In [230]:
result['cut'].value_counts()

Premium    13791
Fair        1610
Name: cut, dtype: int64

In [203]:
def cut_mean(x):
    return x['price'].mean() >= 4000

In [204]:
dia.groupby('cut').filter(cut_mean)

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
8,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49
12,0.22,Premium,F,SI1,60.4,61.0,342,3.88,3.84,2.33
14,0.20,Premium,E,SI2,60.2,62.0,345,3.79,3.75,2.27
15,0.32,Premium,E,I1,60.9,58.0,345,4.38,4.42,2.68
26,0.24,Premium,I,VS1,62.5,57.0,355,3.97,3.94,2.47
45,0.29,Premium,F,SI1,62.4,58.0,403,4.24,4.26,2.65
53,0.22,Premium,E,VS2,61.6,58.0,404,3.93,3.89,2.41
54,0.22,Premium,D,VS2,59.3,62.0,404,3.91,3.88,2.31


In [205]:
# color 별 carat의 최대값과 최소값의 차이가 2이상 3미만인 diamond 데이터들 조회
dia.groupby('color').filter(lambda x : 2<= x['carat'].max() -x['carat'].min() < 3 )

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
8,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49
12,0.22,Premium,F,SI1,60.4,61.0,342,3.88,3.84,2.33
14,0.20,Premium,E,SI2,60.2,62.0,345,3.79,3.75,2.27
15,0.32,Premium,E,I1,60.9,58.0,345,4.38,4.42,2.68
21,0.23,Very Good,E,VS2,63.8,55.0,352,3.85,3.92,2.48
25,0.23,Very Good,G,VVS2,60.4,58.0,354,3.97,4.01,2.41
29,0.23,Very Good,F,VS1,60.9,57.0,357,3.96,3.99,2.42


In [292]:
k=dia.groupby('color')['carat'].max().sort_index()-dia.groupby('color')['carat'].min().sort_index()
k  # E,F,G,  # sort_index() 해주기// 혹시 인덱스가 다를 수 있으니까 정렬해놓고 연산 

color
D    3.20
E    2.85
F    2.81
G    2.78
H    3.90
I    3.78
J    4.78
Name: carat, dtype: float64

In [216]:
# color 별 carat의 최대값과 최소값의 차이가 2이상 3미만인 diamond 데이터들 조회
def carat_max_min(x):
    return 2 <= x['carat'].max()-x['carat'].min() <3  # 다른 파이썬 패키지에서는 이런 문법지원 안할 수도 있어서 강사님이 한것으로 

In [218]:
dia.groupby('color').filter(carat_max_min)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
8,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49
12,0.22,Premium,F,SI1,60.4,61.0,342,3.88,3.84,2.33
14,0.20,Premium,E,SI2,60.2,62.0,345,3.79,3.75,2.27
15,0.32,Premium,E,I1,60.9,58.0,345,4.38,4.42,2.68
21,0.23,Very Good,E,VS2,63.8,55.0,352,3.85,3.92,2.48
25,0.23,Very Good,G,VVS2,60.4,58.0,354,3.97,4.01,2.41
29,0.23,Very Good,F,VS1,60.9,57.0,357,3.96,3.99,2.42


In [295]:
dia.groupby('cut')['price'].max()

cut
Fair         18574
Good         18788
Ideal        18806
Premium      18823
Very Good    18818
Name: price, dtype: int64

In [231]:
def carat_diff(x):
    diff = x['carat'].max()-x['carat'].min()
    return (diff >=2) & (diff<=3)

In [234]:
result = dia.groupby('color').filter(carat_diff)
result

,carat,cut,color,clarity,clarity_mean_price,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,NaN,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,NaN,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,NaN,56.9,65.0,327,4.05,4.07,2.31
8,0.22,Fair,E,VS2,NaN,65.1,61.0,337,3.87,3.78,2.49
12,0.22,Premium,F,SI1,NaN,60.4,61.0,342,3.88,3.84,2.33
14,0.20,Premium,E,SI2,NaN,60.2,62.0,345,3.79,3.75,2.27
15,0.32,Premium,E,I1,NaN,60.9,58.0,345,4.38,4.42,2.68
21,0.23,Very Good,E,VS2,NaN,63.8,55.0,352,3.85,3.92,2.48
25,0.23,Very Good,G,VVS2,NaN,60.4,58.0,354,3.97,4.01,2.41
29,0.23,Very Good,F,VS1,NaN,60.9,57.0,357,3.96,3.99,2.42


In [235]:
result['color'].value_counts()

G    11292
E     9797
F     9542
Name: color, dtype: int64

## a모르겟음

In [237]:
# clarity 별 평균 가격 컬럼을 DataFrame에 추가.
np.round(dia.groupby('clarity')['price'].mean(),2)


clarity
I1      3924.17
IF      2864.84
SI1     3996.00
SI2     5063.03
VS1     3839.46
VS2     3924.99
VVS1    2523.11
VVS2    3283.74
Name: price, dtype: float64

In [223]:
dia.info() # 7번자리에 넣기 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.1+ MB


In [240]:
mean_price = np.round(dia.groupby('clarity')['price'].transform('mean'),2)
mean_price # transform 을 안 했었다...

0        5063.03
1        3996.00
2        3839.46
3        3924.99
4        5063.03
5        3283.74
6        2523.11
7        3996.00
8        3924.99
9        3839.46
10       3996.00
11       3839.46
12       3996.00
13       5063.03
14       5063.03
15       3924.17
16       5063.03
17       3996.00
18       3996.00
19       3996.00
20       5063.03
21       3924.99
22       3839.46
23       3996.00
24       3996.00
25       3283.74
26       3839.46
27       3924.99
28       3924.99
29       3839.46
          ...   
53910    3996.00
53911    2864.84
53912    2523.11
53913    3924.99
53914    3839.46
53915    5063.03
53916    3996.00
53917    3996.00
53918    3839.46
53919    2523.11
53920    3924.99
53921    3924.99
53922    3839.46
53923    3924.99
53924    3924.99
53925    3996.00
53926    3996.00
53927    3996.00
53928    5063.03
53929    3839.46
53930    3996.00
53931    3996.00
53932    3924.99
53933    3924.99
53934    3996.00
53935    3996.00
53936    3996.00
53937    3996.

In [243]:
dia2 = dia.copy()
dia2.insert(7,'mean_price_clarity',mean_price)  # 위에서 transform 을 안 했었다...

ValueError: cannot insert mean_price_clarity, already exists

In [255]:
#dia2= dia2.drop('clarity_mean_price',axis =1)
dia2

,carat,cut,color,clarity,depth,table,mean_price_clarity,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,5063.03,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,3996.00,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,3839.46,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,3924.99,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,5063.03,335,4.34,4.35,2.75
5,0.24,Very Good,J,VVS2,62.8,57.0,3283.74,336,3.94,3.96,2.48
6,0.24,Very Good,I,VVS1,62.3,57.0,2523.11,336,3.95,3.98,2.47
7,0.26,Very Good,H,SI1,61.9,55.0,3996.00,337,4.07,4.11,2.53
8,0.22,Fair,E,VS2,65.1,61.0,3924.99,337,3.87,3.78,2.49
9,0.23,Very Good,H,VS1,59.4,61.0,3839.46,338,4.00,4.05,2.39


In [251]:
dia=dia.drop('mean_price_clarity',axis=1)
dia

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
5,0.24,Very Good,J,VVS2,62.8,57.0,336,3.94,3.96,2.48
6,0.24,Very Good,I,VVS1,62.3,57.0,336,3.95,3.98,2.47
7,0.26,Very Good,H,SI1,61.9,55.0,337,4.07,4.11,2.53
8,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49
9,0.23,Very Good,H,VS1,59.4,61.0,338,4.00,4.05,2.39


# pivot_table()
엑셀의 pivot table 기능을 제공하는 메소드.    
분류별 집계(Group으로 묶어 집계)를 처리하는 함수로 group으로 묶고자 하는 컬럼을 행과 열로 위치시키고 집계값을 값으로 보여준다.    
역할은 groupby() 를 이용한 집계와 같다.

> pivot() 함수와 역할이 다르다.   
> pivot() 은 index와 column의 형태를 바꾸는 reshape 함수.
> pivot_table() 은 집계함수 
- `DataFrame.pivot_table(values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')`

-values=None, index=None, columns=None 반드시,


- **매개변수**
    - index
        - 문자열 또는 리스트. index로 올 컬럼들 => groupby였으면 묶었을 컬럼
    - columns
        - 문자열 또는 리스트. column으로 올 컬럼들 => groupby였으면 묶었을 컬럼 (index/columns가 묶여서 groupby에 묶을 컬럼들이 된다.)
    - values
        - 문자열 또는 리스트. <<<<<집계할 대상 컬럼들>>>
    - aggfunc
        - 집계함수 지정. 함수, 함수이름문자열, 함수리스트(함수이름 문자열/함수객체), dict: 집계할 함수
        - 기본(생략시): 평균을 구한다. (mean이 기본값)
    - fill_value, dropna
        - fill_value: 집계시 NA가 나올경우 채울 값
        - dropna: boolean. 컬럼의 전체값이 NA인 경우 그 컬럼 제거(기본: True)
    - margins/margins_name
        - margins: boolean(기본: False). 총집계결과를 만들지 여부.
        - margins_name: margin의 이름 문자열로 지정 (생략시 All)

In [82]:
import pandas as dp
import numpy as np
flights = pd.read_csv('data/flights.csv')
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58492 entries, 0 to 58491
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MONTH      58492 non-null  int64  
 1   DAY        58492 non-null  int64  
 2   WEEKDAY    58492 non-null  int64  
 3   AIRLINE    58492 non-null  object 
 4   ORG_AIR    58492 non-null  object 
 5   DEST_AIR   58492 non-null  object 
 6   SCHED_DEP  58492 non-null  int64  
 7   DEP_DELAY  57659 non-null  float64
 8   AIR_TIME   57474 non-null  float64
 9   DIST       58492 non-null  int64  
 10  SCHED_ARR  58492 non-null  int64  
 11  ARR_DELAY  57474 non-null  float64
 12  DIVERTED   58492 non-null  int64  
 13  CANCELLED  58492 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 6.2+ MB


In [84]:
flights.groupby(['AIRLINE','WEEKDAY'])['AIR_TIME'].mean()

AIRLINE  WEEKDAY
AA       1          147.610569
         2          143.851852
         3          144.514005
         4          141.124618
         5          145.430966
                       ...    
WN       3          104.219920
         4          107.200800
         5          107.893635
         6          109.247433
         7          107.602273
Name: AIR_TIME, Length: 98, dtype: float64

In [ ]:
x

### 1개의 컬럼을 grouping 해서 집계
- 항공사별 비행시간의 평균 
- 사용컬럼
    - grouping할 컬럼
        - AIRLINE: 항공사
    - 집계대상컬럼
        - AIR_TIME
- 집계: mean

In [85]:
flights.groupby(['AIRLINE'])['AIR_TIME'].mean() # 시리즈로 반환 
#groupby  로 했을때는 그냥 멀티 인덱스로 반환한다. /

AIRLINE
AA    144.259404
AS    147.845052
B6    209.412963
DL    115.334187
EV     68.964016
F9    127.592337
HA    338.288288
MQ     61.318346
NK    135.736878
OO     76.010272
UA    155.650521
US    147.686755
VX    154.864097
WN    107.005897
Name: AIR_TIME, dtype: float64

In [86]:
flights.pivot_table(values = 'AIR_TIME', index='AIRLINE', aggfunc='mean')  # values -> 어떤컬럼의 값을 집계할 것인가?

,AIR_TIME
AIRLINE,
AA,144.259404
AS,147.845052
B6,209.412963
DL,115.334187
EV,68.964016
F9,127.592337
HA,338.288288
MQ,61.318346
NK,135.736878


In [87]:
flights.pivot_table(values = 'AIR_TIME', columns='AIRLINE', aggfunc='mean')

AIRLINE,AA,AS,B6,DL,EV,F9,HA,MQ,NK,OO,UA,US,VX,WN
AIR_TIME,144.259404,147.845052,209.412963,115.334187,68.964016,127.592337,338.288288,61.318346,135.736878,76.010272,155.650521,147.686755,154.864097,107.005897


In [89]:
flights.pivot_table(values = 'AIR_TIME', index='AIRLINE', aggfunc='mean' ,margins_name = 'total mean',margins = True)

,AIR_TIME
AIRLINE,
AA,144.259404
AS,147.845052
B6,209.412963
DL,115.334187
EV,68.964016
F9,127.592337
HA,338.288288
MQ,61.318346
NK,135.736878


In [91]:
flights.pivot_table(values = 'AIR_TIME', index='AIRLINE', aggfunc=['mean','sum','count']) #index='AIRLINE'  --> groupby 

,mean,sum,count
,AIR_TIME,AIR_TIME,AIR_TIME
AIRLINE,,,
AA,144.259404,1257942.0,8720
AS,147.845052,113545.0,768
B6,209.412963,113083.0,540
DL,115.334187,1215507.0,10539
EV,68.964016,392888.0,5697
F9,127.592337,166508.0,1305
HA,338.288288,37550.0,111
MQ,61.318346,203209.0,3314


### 두개의 컬럼을 grouping 해서 집계
- 항공사/출발공항코드 별 취소 총수 (1이 취소이므로 합계를 구한다.)
- 사용컬럼
    - grouping할 컬럼
        - AIRLINE: 항공사
        - ORG_AIR: 출발 공항코드
    - 집계대상컬럼
        - CANCELLED: 취소여부 - 1:취소, 0: 취소안됨
- 집계: sum

In [93]:
flights['ORG_AIR'].unique()

array(['LAX', 'DEN', 'DFW', 'IAH', 'SFO', 'ORD', 'PHX', 'ATL', 'LAS',
       'MSP'], dtype=object)

In [97]:
a = flights.groupby(['AIRLINE','ORG_AIR'])['CANCELLED'].sum()  # 결측치는 아예 빠져서 반환됨 
a

AIRLINE  ORG_AIR
AA       ATL         3
         DEN         4
         DFW        86
         IAH         3
         LAS         3
                    ..
WN       LAS         7
         LAX        32
         MSP         1
         PHX         6
         SFO        25
Name: CANCELLED, Length: 114, dtype: int64

In [98]:
a['AA','ATL']

3

In [100]:
a['HA','ATL']  # error 발생 //HA 항공사는 ATL 출발 한적이 아예 없어서 그룹으로 묶이는 애들이 없다.

KeyError: ('HA', 'ATL')

In [105]:
flights.pivot_table(values = 'CANCELLED',
                    index ='AIRLINE', 
                    columns='ORG_AIR', 
                    aggfunc = 'sum',
                    #fill_value = -1
                   )
# 결측치(NaN)이 결과에 나오는 것은 (행,열) 그룹으로 묶인 데이터가 한개도 없기 때문.
# ex) b6 항공이 ATL 공항에서 출발한게 아예 없다.

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO
AIRLINE,,,,,,,,,,
AA,3.0,4.0,86.0,3.0,3.0,11.0,3.0,35.0,4.0,2.0
AS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B6,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,1.0
DL,28.0,1.0,0.0,0.0,1.0,1.0,4.0,0.0,1.0,2.0
EV,18.0,6.0,27.0,36.0,NaN,NaN,6.0,53.0,0.0,NaN
F9,0.0,2.0,1.0,0.0,1.0,1.0,1.0,4.0,0.0,0.0
HA,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0
MQ,5.0,NaN,62.0,0.0,NaN,0.0,0.0,85.0,NaN,NaN
NK,1.0,1.0,6.0,0.0,1.0,1.0,3.0,10.0,2.0,NaN


In [ ]:
AA,ATL

In [106]:
flights.pivot_table(values = 'CANCELLED',
                    index ='AIRLINE', 
                    columns='ORG_AIR', 
                    aggfunc = 'sum',
                    margins =True,
                    margins_name = 'total sum')

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO,total sum
AIRLINE,,,,,,,,,,,
AA,3.0,4.0,86.0,3.0,3.0,11.0,3.0,35.0,4.0,2.0,154
AS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
B6,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,1.0,1
DL,28.0,1.0,0.0,0.0,1.0,1.0,4.0,0.0,1.0,2.0,38
EV,18.0,6.0,27.0,36.0,NaN,NaN,6.0,53.0,0.0,NaN,146
F9,0.0,2.0,1.0,0.0,1.0,1.0,1.0,4.0,0.0,0.0,10
HA,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0
MQ,5.0,NaN,62.0,0.0,NaN,0.0,0.0,85.0,NaN,NaN,152
NK,1.0,1.0,6.0,0.0,1.0,1.0,3.0,10.0,2.0,NaN,25


In [110]:
flights.pivot_table(values = ['CANCELLED','AIR_TIME'],
                    index ='AIRLINE', 
                    columns='ORG_AIR', 
                    aggfunc = 'sum',
                    #margins =True,
                    #margins_name = 'total sum'
                   )

AIR_TIME                                                              \
ORG_AIR       ATL       DEN       DFW       IAH       LAS       LAX       MSP   
AIRLINE                                                                         
AA        27074.0   24832.0  499575.0   18740.0   54814.0  205473.0   17267.0   
AS         3841.0    7306.0   10426.0    1837.0   15787.0   36277.0    2979.0   
B6            NaN    5065.0    3922.0       NaN   19996.0   36468.0       NaN   
DL       617926.0   30265.0   14886.0    9833.0   51365.0  147066.0  232860.0   
EV       103710.0   13953.0   40386.0  127934.0       NaN       NaN   17328.0   
F9        17138.0   76089.0    4142.0    6421.0   16785.0    5800.0    4737.0   
HA            NaN       NaN       NaN       NaN    7846.0   15952.0       NaN   
MQ         5244.0       NaN   99593.0      91.0       NaN     631.0     235.0   
NK        12023.0   13317.0   35950.0   18000.0   48125.0   20761.0   11830.0   
OO        12915.0  106038.0   13134.0   45807.0    5896.0   74904.0   52957.0   
UA        11064.0  186866.0   10880.0  244750.0   48514.0  166396.0   10545.0   
US         5450.0    8211.0   14209.0    9938.0   14794.0   22919.0   10104.0   
VX            NaN       NaN       NaN       NaN    8853.0   62951.0       NaN   
WN       124302.0  190875.0       NaN       NaN  216185.0  115537.0   21134.0   

                                      CANCELLED                               \
ORG_AIR       ORD       PHX       SFO       ATL   DEN   DFW   IAH  LAS   LAX   
AIRLINE                                                                        
AA       211361.0  127960.0   70846.0       3.0   4.0  86.0   3.0  3.0  11.0   
AS        13270.0    9948.0   11874.0       0.0   0.0   0.0   0.0  0.0   0.0   
B6         9325.0    4678.0   33629.0       NaN   0.0   0.0   NaN  0.0   0.0   
DL        16318.0   37556.0   57432.0      28.0   1.0   0.0   0.0  1.0   1.0   
EV        89444.0     133.0       NaN      18.0   6.0  27.0  36.0  NaN   NaN   
F9        21787.0    6331.0    7278.0       0.0   2.0   1.0   0.0  1.0   1.0   
HA            NaN    7275.0    6477.0       NaN   NaN   NaN   NaN  0.0   0.0   
MQ        97415.0       NaN       NaN       5.0   NaN  62.0   0.0  NaN   0.0   
NK        37433.0    4266.0       NaN       1.0   1.0   6.0   0.0  1.0   1.0   
OO        71114.0   34590.0   71011.0       3.0  25.0   2.0  10.0  0.0  15.0   
UA       237731.0   24298.0  254352.0       2.0   9.0   1.0  23.0  3.0   6.0   
US        11863.0  123327.0   14450.0       0.0   0.0   2.0   2.0  1.0   0.0   
VX        10175.0       NaN   70717.0       NaN   NaN   NaN   NaN  0.0   3.0   
WN            NaN  181817.0   39369.0       9.0  13.0   NaN   NaN  7.0  32.0   

                               
ORG_AIR  MSP   ORD  PHX   SFO  
AIRLINE                        
AA       3.0  35.0  4.0   2.0  
AS       0.0   0.0  0.0   0.0  
B6       NaN   0.0  0.0   1.0  
DL       4.0   0.0  1.0   2.0  
EV       6.0  53.0  0.0   NaN  
F9       1.0   4.0  0.0   0.0  
HA       NaN   NaN  0.0   0.0  
MQ       0.0  85.0  NaN   NaN  
NK       3.0  10.0  2.0   NaN  
OO       4.0  41.0  9.0  33.0  
UA       2.0  25.0  3.0  19.0  
US       0.0   6.0  7.0   3.0  
VX       NaN   0.0  NaN   3.0  
WN       1.0   NaN  6.0  25.0

### 3개 이상의 컬럼을 grouping해서 집계
- 항공사/월/출발공항코드 별 취소 총수 
- grouping할 컬럼
    - AIRLINE:항공사
    - MONTH:월
    - ORG_AIR: 출발지 공항
- 집계 대상컬럼
    - CANCELLED: 취소여부
- 집계 : sum    

In [111]:
flights.groupby(['AIRLINE','MONTH','ORG_AIR'])['CANCELLED'].sum()  # org air 출발 공항

AIRLINE  MONTH  ORG_AIR
AA       1      ATL        0
                DEN        0
                DFW        8
                IAH        0
                LAS        0
                          ..
WN       12     LAS        1
                LAX        2
                MSP        0
                PHX        0
                SFO        0
Name: CANCELLED, Length: 1133, dtype: int64

In [114]:
result = flights.pivot_table(values = 'CANCELLED',
                             index = ['AIRLINE','ORG_AIR'],
                             columns = 'MONTH',
                             aggfunc = 'sum',
                             margins= True,
                             margins_name = 'TOTAL SUM')

# 엑셀 파일로 저장해서 전체 데이터를 한번에 볼수 도 있다.  데이터가 너무 많으면  ... 으로 나오니까 한번에 볼때 

MONTH                  1      2     3     4     5      6     7     8     9  \
AIRLINE   ORG_AIR                                                            
AA        ATL        0.0    2.0   1.0   0.0   0.0    0.0   0.0   0.0   0.0   
          DEN        0.0    1.0   0.0   0.0   2.0    0.0   0.0   0.0   0.0   
          DFW        8.0   33.0  13.0   4.0   8.0    7.0   1.0   2.0   1.0   
          IAH        0.0    1.0   0.0   0.0   1.0    0.0   0.0   1.0   0.0   
          LAS        0.0    2.0   0.0   0.0   0.0    0.0   1.0   0.0   0.0   
...                  ...    ...   ...   ...   ...    ...   ...   ...   ...   
WN        LAX        3.0    2.0   3.0   2.0   1.0    0.0   9.0   4.0   3.0   
          MSP        1.0    0.0   0.0   0.0   0.0    0.0   0.0   0.0   0.0   
          PHX        0.0    2.0   1.0   1.0   0.0    0.0   1.0   0.0   0.0   
          SFO        4.0    5.0   0.0   2.0   2.0    6.0   0.0   2.0   2.0   
TOTAL SUM          104.0  195.0  94.0  58.0  81.0  100.0  56.0  44.0  17.0   

MONTH                11    12  TOTAL SUM  
AIRLINE   ORG_AIR                         
AA        ATL       0.0   0.0          3  
          DEN       0.0   1.0          4  
          DFW       3.0   6.0         86  
          IAH       0.0   0.0          3  
          LAS       0.0   0.0          3  
...                 ...   ...        ...  
WN        LAX       3.0   2.0         32  
          MSP       0.0   0.0          1  
          PHX       1.0   0.0          6  
          SFO       2.0   0.0         25  
TOTAL SUM          51.0  81.0        881  

[115 rows x 12 columns]

### 3개 이상의 컬럼을 grouping해서 집계 2
- 항공사/월/출발공항코드 별 최대/최소 연착시간
- grouping할 컬럼
    - AIRLINE:항공사
    - MONTH:월
    - ORG_AIR: 출발지 공항
- 집계 대상컬럼
    - ARR_DELAY: 연착시간
- 집계 : min, max    

In [123]:
pd.options.display.min_rows = 100  # ch
#pd.options.display.min_width = 1000

In [124]:
flights.pivot_table(values = 'ARR_DELAY',
                    index = ['AIRLINE','ORG_AIR'],
                    columns = 'MONTH',
                    aggfunc = ['min','max','count'])

min                                                        \
MONTH              1     2     3     4     5     6     7     8     9     11   
AIRLINE ORG_AIR                                                               
AA      ATL     -27.0 -26.0 -32.0 -30.0 -26.0 -23.0 -27.0 -32.0 -33.0 -33.0   
        DEN     -13.0 -17.0 -19.0 -28.0 -20.0 -13.0 -30.0 -27.0 -27.0 -27.0   
        DFW     -39.0 -29.0 -29.0 -37.0 -36.0 -33.0 -32.0 -32.0 -33.0 -45.0   
        IAH     -23.0 -27.0 -13.0 -27.0 -19.0 -13.0 -19.0 -30.0 -31.0 -14.0   
        LAS     -32.0 -25.0 -24.0 -19.0 -31.0 -27.0 -40.0 -28.0 -32.0 -31.0   
        LAX     -42.0 -41.0 -31.0 -40.0 -41.0 -39.0 -42.0 -49.0 -38.0 -37.0   
        MSP     -34.0 -32.0 -38.0 -10.0 -15.0 -17.0 -26.0 -22.0 -28.0 -33.0   
        ORD     -38.0 -34.0 -34.0 -52.0 -32.0 -43.0 -35.0 -36.0 -46.0 -50.0   
        PHX     -15.0 -16.0 -24.0 -22.0 -17.0 -26.0 -26.0 -26.0 -27.0 -35.0   
        SFO     -20.0 -31.0 -31.0 -33.0 -45.0 -33.0 -46.0 -39.0 -38.0 -29.0   
AS      ATL       NaN  -5.0 -15.0  -5.0 -44.0   NaN   NaN  69.0   NaN -13.0   
        DEN     -17.0 -21.0 -28.0 -14.0 -13.0 -12.0 -25.0 -22.0  -1.0 -38.0   
        DFW     -29.0 -23.0 -38.0 -19.0 -27.0 -29.0 -13.0 -10.0 -20.0 -23.0   
        IAH       8.0   NaN   NaN -10.0  23.0   4.0  -6.0   NaN   1.0   NaN   
        LAS     -31.0 -35.0 -30.0 -35.0 -13.0 -20.0 -26.0 -20.0 -16.0 -31.0   
        LAX     -21.0 -45.0 -22.0 -34.0 -22.0 -30.0 -21.0 -21.0 -28.0 -35.0   
        MSP       5.0   NaN  14.0   NaN  -4.0  -8.0   NaN   NaN   7.0 -23.0   
        ORD     -28.0  -7.0 -12.0 -26.0 -27.0 -24.0 -21.0 -20.0 -14.0   2.0   
        PHX     -57.0 -34.0 -12.0 -23.0 -23.0 -20.0 -13.0 -26.0 -21.0 -33.0   
        SFO     -28.0  -8.0 -25.0 -34.0 -25.0 -28.0 -28.0 -22.0 -18.0 -16.0   
B6      DEN      -4.0   0.0 -11.0 -12.0 -26.0  22.0   NaN -19.0 -14.0 -17.0   
        DFW      -1.0   9.0   NaN  -4.0 -21.0 -15.0   NaN  63.0 -14.0  -2.0   
        LAS     -27.0 -15.0 -24.0 -24.0 -23.0 -28.0 -45.0 -39.0 -20.0 -23.0   
        LAX     -15.0 -22.0  -8.0 -23.0 -43.0 -37.0 -33.0 -51.0 -27.0 -29.0   
        ORD     -22.0  29.0 -27.0 -24.0 -39.0  -3.0 -31.0 -20.0 -15.0 -13.0   
        PHX      -5.0 -13.0  -9.0  -8.0   NaN   NaN   NaN   NaN   NaN -11.0   
        SFO     -26.0 -28.0 -37.0 -26.0 -31.0 -29.0 -38.0 -37.0 -26.0 -24.0   
DL      ATL     -37.0 -33.0 -35.0 -53.0 -32.0 -33.0 -38.0 -37.0 -33.0 -57.0   
        DEN     -36.0 -16.0 -29.0 -22.0 -31.0 -34.0 -21.0 -23.0 -20.0 -27.0   
        DFW     -31.0 -14.0 -15.0 -23.0 -15.0 -20.0 -27.0 -26.0 -24.0 -21.0   
...               ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
UA      DEN     -33.0 -34.0 -34.0 -32.0 -31.0 -41.0 -39.0 -40.0 -42.0 -39.0   
        DFW     -23.0  -9.0 -30.0 -26.0 -16.0 -22.0 -26.0 -33.0 -29.0 -24.0   
        IAH     -33.0 -41.0 -47.0 -39.0 -35.0 -39.0 -37.0 -30.0 -27.0 -38.0   
        LAS     -23.0 -33.0 -28.0 -38.0 -48.0 -36.0 -34.0 -29.0 -34.0 -31.0   
        LAX     -26.0 -58.0 -29.0 -48.0 -41.0 -36.0 -34.0 -34.0 -48.0 -48.0   
        MSP     -25.0 -21.0 -24.0 -22.0 -22.0 -26.0 -26.0 -23.0 -15.0 -30.0   
        ORD     -37.0 -28.0 -43.0 -31.0 -36.0 -48.0 -32.0 -34.0 -36.0 -38.0   
        PHX     -32.0 -19.0 -26.0 -42.0 -38.0 -27.0 -34.0 -32.0 -21.0 -39.0   
        SFO     -34.0 -39.0 -37.0 -45.0 -42.0 -32.0 -37.0 -40.0 -35.0 -39.0   
US      ATL     -46.0 -10.0 -41.0 -24.0 -33.0 -23.0   NaN   NaN   NaN   NaN   
        DEN     -17.0 -14.0 -25.0 -25.0 -15.0 -28.0   NaN   NaN   NaN   NaN   
        DFW     -19.0 -32.0 -24.0 -20.0 -23.0  -8.0   NaN   NaN   NaN   NaN   
        IAH     -17.0 -21.0 -21.0  -8.0 -16.0 -18.0   NaN   NaN   NaN   NaN   
        LAS     -26.0 -23.0 -20.0 -48.0 -40.0 -27.0   NaN   NaN   NaN   NaN   
        LAX     -13.0 -19.0 -27.0 -39.0 -51.0 -41.0   NaN   NaN   NaN   NaN   
        MSP     -24.0 -35.0 -33.0 -20.0 -21.0 -20.0   NaN   NaN   NaN   NaN   
        ORD     -30.0 -17.0 -20.0 -37.0 -31.0 -50.0   NaN   NaN   Na

---------------------------- 위에까진 groupby 와 연관된 메서드 --------------------------------------------------------------

# apply() - Series, DataFrame의 데이터 일괄 처리

데이터프레임의 행들과 열들 또는 Series의 원소들에 공통된 처리를 할 때 apply 함수를 이용하면 반복문을 사용하지 않고 일괄 처리가 가능하다.

- DataFrame.apply(함수, axis=0, args=())
    - 인수로 행이나 열을 받는 함수를 apply 메서드의 인수로 넣으면 데이터프레임의 행이나 열들을 하나씩 함수에 전달한다.
    - 매개변수
        - 함수: DataFrame의 행들 또는 열들을 전달할 함수
        - axis: **0-열을 전달, 1-행을 전달 (기본값 0)** G: 0이 행이다...
        - args: 행/열 이외에 전달할 매개변수를 위치기반(순서대로) 튜플로 전달
- Series.apply(함수, args=())
    - 인수로 Series의 원소들을 받는 함수를 apply 메소드의 인수로 넣으면  Series의 원소들을 하나씩 함수로 전달한다.
    - 매개변수
        - 함수: Series의 원소들을 전달할 함수
        - args: 원소 이외에 전달할 매개변수를 위치기반(순서대로) 튜플로 전달

In [264]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.arange(24).reshape(6,4), columns = ['NO1','NO2','NO3','NO4'])
df

,NO1,NO2,NO3,NO4
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23


In [133]:
print(df**2)
df

   NO1  NO2  NO3  NO4
0    0    1    4    9
1   16   25   36   49
2   64   81  100  121
3  144  169  196  225
4  256  289  324  361
5  400  441  484  529


,NO1,NO2,NO3,NO4
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23


In [135]:
def func(x):
    '''
    apply() 에 적용할 함수
    [매개변수]
         x : DataFrame 에 적용시 Series , Series에 정용시 - int scalar
    [반환값]
         x를 처리한 결과
    '''
    print(type(x))
    return x**2


In [136]:
df.apply(func) #함수가 4번 호출 됨  데이터 프레임에서 각칼럼이 series 로 전달되서 벡터화연산이 일어남 

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


,NO1,NO2,NO3,NO4
0,0,1,4,9
1,16,25,36,49
2,64,81,100,121
3,144,169,196,225
4,256,289,324,361
5,400,441,484,529


In [137]:
# Series
df['NO1'].apply(func)  # Series 에서 x 에전달되는 값이 원소

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


0      0
1     16
2     64
3    144
4    256
5    400
Name: NO1, dtype: int64

In [138]:
df.apply(lambda x : x**2)

,NO1,NO2,NO3,NO4
0,0,1,4,9
1,16,25,36,49
2,64,81,100,121
3,144,169,196,225
4,256,289,324,361
5,400,441,484,529


In [139]:
df.apply(func,axis=1)  # DataFrame 은 행,열 방향으로 Series 를 넘겨줄 수있다

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


,NO1,NO2,NO3,NO4
0,0,1,4,9
1,16,25,36,49
2,64,81,100,121
3,144,169,196,225
4,256,289,324,361
5,400,441,484,529


In [142]:
df

,NO1,NO2,NO3,NO4
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23


In [140]:
df.apply(lambda x : x.mean())

NO1    10.0
NO2    11.0
NO3    12.0
NO4    13.0
dtype: float64

In [261]:
df

,NO1,NO2,NO3,NO4
0,0,1,2,3원
1,4,5,6,7원
2,8,9,10,11원
3,12,13,14,15원
4,16,17,18,19원
5,20,21,22,23원


In [260]:
df['NO4']=df['NO4'].apply(lambda x : f'{x}원')

In [259]:
df['NO4'].apply(lambda x : f'{x}원')  # 

0     3원
1     7원
2    11원
3    15원
4    19원
5    23원
Name: NO4, dtype: object

In [150]:
df

,NO1,NO2,NO3,NO4
0,0,1,2,3원원원
1,4,5,6,7원원원
2,8,9,10,11원원원
3,12,13,14,15원원원
4,16,17,18,19원원원
5,20,21,22,23원원원


In [ ]:
pd.cut

# cut()/qcut() - 연속형(실수)을 범주형으로 변환
- cut() : 지정한 값을 기준으로 구간을 나눠 그룹으로 묶는다.
    - `pd.cut(x, bins,right=True, labels=None)`
    - 매개변수
        - x: 나눌 대상. 1차원 배열형태의 자료구조
        - bins: 나누는 기준값(구간경계)을 리스트로 전달
        - right: 구간경계의 오른쪽(True)을 포함할지 왼쪽(False)을 포함할지
        - labels: 각 구간의 label을 리스트로 전달
- qcut() : 데이터를 오름차순으로 정렬한 뒤 데이터 개수가 같도록 **지정한 개수**만큼의 구간으로 나눈다.
    - `pd.qcut(x, q, labels)`
    - 매개변수
        - x: 나눌 대상. 1차원 배열형태의 자료구조
        - q: 나눌 개수

In [154]:
ages = pd.Series(np.random.randint(50, size = 30))
ages.shape

(30,)

In [155]:
ages.value_counts()

12    2
3     2
38    2
9     2
29    2
11    1
33    1
34    1
39    1
37    1
7     1
8     1
10    1
41    1
1     1
48    1
44    1
19    1
21    1
22    1
5     1
24    1
25    1
28    1
0     1
dtype: int64

In [158]:
for i in ages:
    print(i,end=' ')

10 11 7 3 37 24 28 12 33 39 19 8 22 21 38 25 12 38 9 1 34 0 29 48 5 44 29 41 3 9 

In [266]:
age_cate = pd.cut(ages,bins = bins)  # 이렇게 되었으니 그룹으로 나눌  수 있다.
age_cate

0     (-1, 10]
1     (10, 20]
2     (-1, 10]
3     (-1, 10]
4     (30, 40]
5     (20, 30]
6     (20, 30]
7     (10, 20]
8     (30, 40]
9     (30, 40]
10    (10, 20]
11    (-1, 10]
12    (20, 30]
13    (20, 30]
14    (30, 40]
15    (20, 30]
16    (10, 20]
17    (30, 40]
18    (-1, 10]
19    (-1, 10]
20    (30, 40]
21    (-1, 10]
22    (20, 30]
23    (40, 51]
24    (-1, 10]
25    (40, 51]
26    (20, 30]
27    (40, 51]
28    (-1, 10]
29    (-1, 10]
dtype: category
Categories (5, interval[int64]): [(-1, 10] < (10, 20] < (20, 30] < (30, 40] < (40, 51]]

In [262]:
bins = [-1,10, 20, 30, 40,51 ]  #bins: 나누는 기준값(구간경계)을 리스트로 전달
age_cate = pd.cut(ages,bins = bins)#, right = False)        # 각각의 값이 어느 범주에 속하는지 범주를 반환함 
print(age_cate.shape)
print(age_cate.value_counts().sort_index())
#() 포함 안함
#[] 포함
#(10,20]  10 < <=20
#[10,20)  10 <= <20 

(30,)
(-1, 10]    10
(10, 20]     4
(20, 30]     7
(30, 40]     6
(40, 51]     3
dtype: int64


In [184]:
for i in ages:
    print(i,end=' ')

10 11 7 3 37 24 28 12 33 39 19 8 22 21 38 25 12 38 9 1 34 0 29 48 5 44 29 41 3 9 

In [185]:
labels = ['10세이하','10대','20대','30대','40대']  # 범주명 지정
age_cate = pd.cut(ages,bins = bins, right = False, labels =labels)  #pd.cut(x, bins,right=True, labels=None)

In [186]:
print(age_cate)

0       10대
1       10대
2     10세이하
3     10세이하
4       30대
5       20대
6       20대
7       10대
8       30대
9       30대
10      10대
11    10세이하
12      20대
13      20대
14      30대
15      20대
16      10대
17      30대
18    10세이하
19    10세이하
20      30대
21    10세이하
22      20대
23      40대
24    10세이하
25      40대
26      20대
27      40대
28    10세이하
29    10세이하
dtype: category
Categories (5, object): ['10세이하' < '10대' < '20대' < '30대' < '40대']


In [182]:
print(ages)

0     10
1     11
2      7
3      3
4     37
5     24
6     28
7     12
8     33
9     39
10    19
11     8
12    22
13    21
14    38
15    25
16    12
17    38
18     9
19     1
20    34
21     0
22    29
23    48
24     5
25    44
26    29
27    41
28     3
29     9
dtype: int32


In [ ]:
# 딕셔너리로 데이터 프레임 생성 을 이용해서 밑에 DataFrame 을 만든 것이다.
data_dict = {
    "id":['id-1','id-5','id-2','id-4','id-3'],
    "korean":[100,50,70,60,90],
    "english":[90,80,100,50,100]  # 데이터로 들어갈 값들의 원소 개수가같아야함 
}
grade = pd.DataFrame(data_dict)
grade
# index명 , key :컬럼명
# 원소 가 같아야한다

In [175]:
df=pd.DataFrame({'나이':ages,'나이대':age_cate})
df

,나이,나이대
0,10,10대
1,11,10대
2,7,10세이하
3,3,10세이하
4,37,30대
5,24,20대
6,28,20대
7,12,10대
8,33,30대
9,39,30대


In [176]:
df.groupby('나이대')['나이'].mean()

나이대
10세이하     5.000000
10대      12.800000
20대      25.428571
30대      36.500000
40대      44.333333
Name: 나이, dtype: float64

In [177]:
dk=pd.qcut(ages, 3, labels = ['범주1','범주2','범주3']) # 3개로 범주를 나눈다./ 최대값 최소값 사이를  3개로 나눈다.
dk

0     범주1
1     범주2
2     범주1
3     범주1
4     범주3
5     범주2
6     범주2
7     범주2
8     범주3
9     범주3
10    범주2
11    범주1
12    범주2
13    범주2
14    범주3
15    범주2
16    범주2
17    범주3
18    범주1
19    범주1
20    범주3
21    범주1
22    범주2
23    범주3
24    범주1
25    범주3
26    범주2
27    범주3
28    범주1
29    범주1
dtype: category
Categories (3, object): ['범주1' < '범주2' < '범주3']

In [268]:
dia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.1+ MB


In [270]:
print(dia.price.min(), dia.price.max())  # 605ㄷ달러짜리 132개
dia.price.value_counts()

326 18823


605      132
802      127
625      126
828      125
776      124
789      121
698      121
544      120
666      114
552      113
720      107
596      107
645      104
526      101
675       99
844       98
1013      98
765       95
684       91
561       91
505       90
984       87
658       86
863       86
743       86
854       85
827       84
945       80
942       79
579       78
        ... 
15695      1
15031      1
16188      1
10933      1
1416       1
1256       1
5514       1
7563       1
9612       1
11661      1
13710      1
3866       1
5977       1
3930       1
16220      1
2769       1
4818       1
10079      1
18291      1
8916       1
13582      1
11533      1
9484       1
3962       1
14205      1
13550      1
13014      1
6811       1
5354       1
11600      1
Name: price, Length: 11602, dtype: int64

In [274]:
# 가격 : 저가 , 중가, 고가 //# . 표기법함  # dia.price
price_cate = pd.cut(dia.price,bins = [300, 1000, 5000, 20000], labels = ['저가', '중가', '고가'])  # 범주(범위)를 만들어줌
price_cate

0        저가
1        저가
2        저가
3        저가
4        저가
5        저가
6        저가
7        저가
8        저가
9        저가
10       저가
11       저가
12       저가
13       저가
14       저가
15       저가
16       저가
17       저가
18       저가
19       저가
20       저가
21       저가
22       저가
23       저가
24       저가
25       저가
26       저가
27       저가
28       저가
29       저가
         ..
53910    중가
53911    중가
53912    중가
53913    중가
53914    중가
53915    중가
53916    중가
53917    중가
53918    중가
53919    중가
53920    중가
53921    중가
53922    중가
53923    중가
53924    중가
53925    중가
53926    중가
53927    중가
53928    중가
53929    중가
53930    중가
53931    중가
53932    중가
53933    중가
53934    중가
53935    중가
53936    중가
53937    중가
53938    중가
53939    중가
Name: price, Length: 53940, dtype: category
Categories (3, object): ['저가' < '중가' < '고가']

In [297]:
price_cate2 = pd.cut(dia.price, 3, labels = ['저가', '중가', '고가'], retbins = True)
price_cate2[0]

0        저가
1        저가
2        저가
3        저가
4        저가
5        저가
6        저가
7        저가
8        저가
9        저가
10       저가
11       저가
12       저가
13       저가
14       저가
15       저가
16       저가
17       저가
18       저가
19       저가
20       저가
21       저가
22       저가
23       저가
24       저가
25       저가
26       저가
27       저가
28       저가
29       저가
         ..
53910    저가
53911    저가
53912    저가
53913    저가
53914    저가
53915    저가
53916    저가
53917    저가
53918    저가
53919    저가
53920    저가
53921    저가
53922    저가
53923    저가
53924    저가
53925    저가
53926    저가
53927    저가
53928    저가
53929    저가
53930    저가
53931    저가
53932    저가
53933    저가
53934    저가
53935    저가
53936    저가
53937    저가
53938    저가
53939    저가
Name: price, Length: 53940, dtype: category
Categories (3, object): ['저가' < '중가' < '고가']

In [282]:
price_cate2[1]

array([  307.503     ,  6491.66666667, 12657.33333333, 18823.        ])

In [285]:
dia['price_cate']= price_cate  # 테이블에 추가함 ---> 집계함수를 사용할때 사용할 수 있음 

In [288]:
dia.tail()

,carat,cut,color,clarity,depth,table,price,x,y,z,price_cate
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50,중가
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61,중가
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56,중가
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74,중가
53939,0.75,Ideal,D,SI2,62.2,55.0,2757,5.83,5.87,3.64,중가


In [289]:
# 가격대별 XXXXX 집계

dia.groupby('price_cate')['carat'].mean()

price_cate
저가    0.334937
중가    0.712814
고가    1.397875
Name: carat, dtype: float64

In [290]:
# 가격대, cut별 가격의 평균 # fair 이면서 평균 4358// 저가의 평균가격 (708)// 전체평균  3932
dia.pivot_table(values = 'price', index ='cut', columns='price_cate', aggfunc = 'mean',margins =True)

price_cate,저가,중가,고가,All
cut,,,,
Fair,776.720721,2845.605976,9074.158879,4358.757764
Good,644.526079,2915.155503,8957.215149,3928.864452
Ideal,733.002773,2362.558209,9335.894082,3457.541970
Premium,734.418851,2729.378705,9507.518974,4584.257704
Very Good,650.268946,2793.512835,9169.034999,3981.759891
All,708.384880,2623.613796,9313.451611,3932.799722


In [ ]:
dia.groupby('pirce')['carat'].mean()  #연속형 값